In [6]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.data.data_dl import data_download_from_ccxt
from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.plotting.plotting_main import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

# prices = data_download_from_ccxt(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-04-15T00:00:00Z',
#     symbols=['BTCUSDT', 'ETHUSD'],
#     timeframe='30m',
# )

# prices.to_hdf("yt_prices.hd5", key='data')
prices = pd.read_hdf('yt_prices.hd5')
rsi_ind = from_talib(
    func_name="rsi",
    prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = is_below(
    want_to_evaluate=rsi_ind,
    user_args=[40, 50],
    cand_ohlc="close",
)

In [7]:
prices

symbol                BTCUSDT                                 ETHUSD           \
candle_info              open      high       low     close     open     high   
open_time                                                                       
2020-10-01 00:00:00 10,759.00 10,817.50 10,759.00 10,790.00   359.85   361.45   
2020-10-01 00:30:00 10,790.00 10,790.50 10,775.00 10,777.00   358.95   359.75   
2020-10-01 01:00:00 10,777.00 10,793.00 10,777.00 10,793.00   359.25   359.80   
2020-10-01 01:30:00 10,793.00 10,836.00 10,792.50 10,822.00   359.75   361.40   
2020-10-01 02:00:00 10,822.00 10,841.50 10,808.00 10,808.50   361.20   364.05   
...                       ...       ...       ...       ...      ...      ...   
2021-04-14 21:30:00 62,703.50 62,812.50 62,615.50 62,795.00 2,406.20 2,440.00   
2021-04-14 22:00:00 62,795.00 62,825.00 62,591.00 62,680.00 2,434.05 2,445.00   
2021-04-14 22:30:00 62,680.00 62,993.00 62,680.00 62,849.50 2,427.00 2,438.60   
2021-04-14 23:00:00 62,849.50 62,999.00 62,750.00 62,999.00 2,431.30 2,450.00   
2021-04-14 23:30:00 62,999.00 62,999.00 62,606.00 62,921.00 2,445.95 2,445.95   

symbol                                 
candle_info              low    close  
open_time                              
2020-10-01 00:00:00   358.85   358.95  
2020-10-01 00:30:00   358.95   359.25  
2020-10-01 01:00:00   358.80   359.75  
2020-10-01 01:30:00   359.75   361.20  
2020-10-01 02:00:00   361.20   362.95  
...                      ...      ...  
2021-04-14 21:30:00 2,397.20 2,434.05  
2021-04-14 22:00:00 2,418.65 2,427.00  
2021-04-14 22:30:00 2,423.50 2,431.30  
2021-04-14 23:00:00 2,431.30 2,445.95  
2021-04-14 23:30:00 2,417.40 2,438.20  

[9408 rows x 8 columns]

In [17]:
rsi_ind

symbol              BTCUSDT       ETHUSD      
RSI_timeperiod           15    30     15    30
open_time                                     
2020-10-01 00:00:00     NaN   NaN    NaN   NaN
2020-10-01 00:30:00     NaN   NaN    NaN   NaN
2020-10-01 01:00:00     NaN   NaN    NaN   NaN
2020-10-01 01:30:00     NaN   NaN    NaN   NaN
2020-10-01 02:00:00     NaN   NaN    NaN   NaN
...                     ...   ...    ...   ...
2021-04-14 21:30:00   45.28 47.78  66.12 62.18
2021-04-14 22:00:00   43.88 46.99  64.17 61.16
2021-04-14 22:30:00   46.50 48.29  64.85 61.55
2021-04-14 23:00:00   48.76 49.41  67.11 62.89
2021-04-14 23:30:00   47.64 48.84  64.75 61.72

[9408 rows x 4 columns]

In [8]:
entries

symbol              BTCUSDT                      ETHUSD                     
RSI_timeperiod           15            30            15            30       
RSI_is_below             40     50     40     50     40     50     40     50
open_time                                                                   
2020-10-01 00:00:00   False  False  False  False  False  False  False  False
2020-10-01 00:30:00   False  False  False  False  False  False  False  False
2020-10-01 01:00:00   False  False  False  False  False  False  False  False
2020-10-01 01:30:00   False  False  False  False  False  False  False  False
2020-10-01 02:00:00   False  False  False  False  False  False  False  False
...                     ...    ...    ...    ...    ...    ...    ...    ...
2021-04-14 21:30:00   False   True  False   True  False  False  False  False
2021-04-14 22:00:00   False   True  False   True  False  False  False  False
2021-04-14 22:30:00   False   True  False   True  False  False  False  False
2021-04-14 23:00:00   False   True  False   True  False  False  False  False
2021-04-14 23:30:00   False   True  False   True  False  False  False  False

[9408 rows x 8 columns]

In [9]:
strat_results_df, settings_results_df = backtest_df_only(
    prices=prices,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(6, 8.1, 1),
    risk_rewards=np.arange(2, 5.1, 0.5),
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(6, 9.1, 0.5),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(6, 9.1, 0.5),
    tsl_when_pct_from_avg_entry=np.arange(2, 4.1, 1),
    gains_pct_filter=150,
    upside_filter=0.6,
    divide_records_array_size_by=2,
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start, I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 2
Total indicator settings per symbol: 4
Total indicator settings to test: 8
Total order settings per symbol: 3,087
Total order settings to test: 6,174
Total candles per symbol: 9,408
Total candles to test: 232,339,968

Total combinations to test: 24,696


In [10]:
strat_results_df

,symbol,entries_col,or_set,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
10198,ETHUSD,7,2449,52.00,358.35,46.15,0.90,"3,583.46","4,583.46"
9485,ETHUSD,7,1420,52.00,289.17,46.15,0.90,"2,891.71","3,891.71"
9484,ETHUSD,7,1419,55.00,274.51,43.64,0.90,"2,745.06","3,745.06"
9050,ETHUSD,7,412,51.00,224.00,47.06,0.90,"2,239.96","3,239.96"
10116,ETHUSD,7,2344,50.00,303.30,44.00,0.89,"3,033.02","4,033.02"
...,...,...,...,...,...,...,...,...,...
954,BTCUSDT,0,1428,42.00,153.38,50.00,0.60,"1,533.77","2,533.77"
3099,BTCUSDT,1,1933,49.00,241.09,48.98,0.60,"2,410.92","3,410.92"
3730,BTCUSDT,1,2778,37.00,162.82,54.05,0.60,"1,628.20","2,628.20"
3820,BTCUSDT,1,2925,37.00,170.78,54.05,0.60,"1,707.85","2,707.85"


In [11]:
settings_results_df.iloc[:,10198]

symbol                            ETHUSD
entries_col                            7
max_equity_risk_pct                 8.00
risk_rewards                        3.00
size_pct                            1.00
tsl_based_on                   low_price
tsl_pcts_init                       8.00
tsl_trail_by_pct                    8.00
tsl_when_pct_from_avg_entry         3.00
Name: 10198, dtype: object

In [12]:
order_records = simulate_up_to_6_nb(
    prices=prices['ETHUSD'].values,
    entries=entries.iloc[:,[7]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr_pct=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=8,
    risk_rewards=3,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_pcts_init=8,
    tsl_trail_by_pct=8,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [15]:
entries.iloc[:,[7]]

symbol,ETHUSD
RSI_timeperiod,30
RSI_is_below,50
open_time,
2020-10-01 00:00:00,False
2020-10-01 00:30:00,False
2020-10-01 01:00:00,False
2020-10-01 01:30:00,False
2020-10-01 02:00:00,False
...,...
2021-04-14 21:30:00,False


In [14]:
rsi_ind[[('ETHUSD', 30)]]

symbol,ETHUSD
RSI_timeperiod,30
open_time,
2020-10-01 00:00:00,NaN
2020-10-01 00:30:00,NaN
2020-10-01 01:00:00,NaN
2020-10-01 01:30:00,NaN
2020-10-01 02:00:00,NaN
...,...
2021-04-14 21:30:00,62.18
2021-04-14 22:00:00,61.16


In [16]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('ETHUSD', 30)]],
        "entries1": entries.iloc[:,[7]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=prices.ETHUSD,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
